# Previsão temporal - preços da bolsa de valores

## Importando bibliotecas

In [3]:
# Importacao desta lib para desativar erro no TensorFlow
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [4]:
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
import sklearn

In [5]:
np.__version__, pd.__version__, tf.__version__, matplotlib.__version__, sklearn.__version__

('1.26.4', '2.2.2', '2.16.1', '3.8.4', '1.4.2')

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

## Carregamento base de dados

In [11]:
base = pd.read_csv('petr4-treinamento.csv')
base

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,19.990000,20.209999,19.690001,19.690001,18.086271,30182600.0
1,2013-01-03,19.809999,20.400000,19.700001,20.400000,18.738441,30552600.0
2,2013-01-04,20.330000,20.620001,20.170000,20.430000,18.766001,36141000.0
3,2013-01-07,20.480000,20.670000,19.950001,20.080000,18.444506,28069600.0
4,2013-01-08,20.110001,20.230000,19.459999,19.500000,17.911745,29091300.0
...,...,...,...,...,...,...,...
1240,2017-12-25,15.750000,15.750000,15.750000,15.750000,15.718563,0.0
1241,2017-12-26,15.750000,15.990000,15.690000,15.970000,15.938125,22173100.0
1242,2017-12-27,15.990000,16.139999,15.980000,16.049999,16.017963,23552200.0
1243,2017-12-28,16.100000,16.129999,16.000000,16.100000,16.067865,19011500.0


## Verificar se há dados faltantes na base de dados

In [12]:
base.isnull().sum()

Date         0
Open         3
High         3
Low          3
Close        3
Adj Close    3
Volume       3
dtype: int64

In [13]:
base = base.dropna()

In [14]:
base.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [15]:
base.shape

(1242, 7)

In [16]:
base_treinamento = base.iloc[:,1:2].values

In [17]:
base_treinamento

array([[19.99    ],
       [19.809999],
       [20.33    ],
       ...,
       [15.99    ],
       [16.1     ],
       [16.1     ]])

## Normalização dos dados - escalonamento

In [18]:
normalizador = MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)
base_treinamento_normalizada

array([[0.76501938],
       [0.7562984 ],
       [0.78149225],
       ...,
       [0.57122093],
       [0.57655039],
       [0.57655039]])

## Separacao base de previsores e alvo com base nos ultimos 90 registros

In [19]:
base_treinamento_normalizada.shape

(1242, 1)

In [21]:
X = [] # previsores
y = [] # alvo - preco real

for i in range(90,1242): # 90 precos anteriores
    X.append(base_treinamento_normalizada[i-90:i,0])
    y.append(base_treinamento_normalizada[i,0])
    #print(i,i-90)

X,y = np.array(X), np.array(y)

In [24]:
X.shape

(1152, 90)

In [25]:
y.shape

(1152,)

## Estrutura da Rede Neural

In [28]:
regressor = Sequential()

regressor.add(LSTM(units = 100, return_sequences=True, input_shape=(X.shape[1],1)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units=1,activation='linear'))

regressor.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 90, 100)        │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 90, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 90, 50)         │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 90, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 90, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 90, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,451 (435.36 KB)

 Trainable params: 111,451 (435.36 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
regressor.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=['mean_absolute_error'])

In [ ]:
regressor.fit(X,y,epochs=100,batch_size=32)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 118ms/step - loss: 0.0487 - mean_absolute_error: 0.1658
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 120ms/step - loss: 0.0136 - mean_absolute_error: 0.0910
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - loss: 0.0164 - mean_absolute_error: 0.0994
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 102ms/step - loss: 0.0099 - mean_absolute_error: 0.0765
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - loss: 0.0112 - mean_absolute_error: 0.0816
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - loss: 0.0094 - mean_absolute_error: 0.0738
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0.0098 - mean_absolute_error: 0.0773
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - loss: 0.0080 - mean_absolute_error: 0.0674
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - loss: 0.0089 - mean_absolute_error: 0.0733
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 0.0082 - mean_absolute_error: 0.0685
Epoch 11/100
36/36 ━━━━━━